# Mining Facebook: Analyzing Fan Pages, Examining Friendships, and More

This IPython Notebook provides an interactive way to follow along with and explore the numbered examples from [_Mining the Social Web (2nd Edition)_](http://bit.ly/135dHfs). The intent behind this notebook is to reinforce the concepts from the sample code in a fun, convenient, and effective way. This notebook assumes that you are reading along with the book and have the context of the discussion as you work through these exercises.

In the somewhat unlikely event that you've somehow stumbled across this notebook outside of its context on GitHub, [you can find the full source code repository here](http://bit.ly/16kGNyb).

## Copyright and Licensing

You are free to use or adapt this notebook for any purpose you'd like. However, please respect the [Simplified BSD License](https://github.com/ptwobrussell/Mining-the-Social-Web-2nd-Edition/blob/master/LICENSE.txt) that governs its use.

# Facebook API Access

Facebook implements OAuth 2.0 as its standard authentication mechanism, but provides a convenient way for you to get an _access token_ for development purposes, and we'll opt to take advantage of that convenience in this notebook. For details on implementing an OAuth flow with Facebook (all from within IPython Notebook), see the \_AppendixB notebook from the [IPython Notebook Dashboard](/).

For this first example, login to your Facebook account and go to https://developers.facebook.com/tools/explorer/ to obtain and set permissions for an access token that you will need to define in the code cell defining the ACCESS_TOKEN variable below.  

Be sure to explore the permissions that are available by clicking on the "Get Access Token" button that's on the page and exploring all of the tabs available. For example, you will need to set the "friends_likes" option under the "Friends Data Permissions" since this permission is used by the script below but is not a basic permission and is not enabled by default. 

<img src="files/resources/ch02-facebook/images/FB_GraphExplorer_perms.png" width="300px" /><br />

In [1]:
# Copy and paste in the value you just got from the inline frame into this variable and execute this cell.
# Keep in mind that you could have just gone to https://developers.facebook.com/tools/access_token/
# and retrieved the "User Token" value from the Access Token Tool
ACCESS_TOKEN = ''


## Example 1. Making Graph API requests over HTTP

In [2]:
import requests # pip install requests
import json

base_url = 'https://graph.facebook.com/me'

# Get 10 likes for 10 friends
fields = 'id,name,friends.limit(10).fields(likes.limit(10))'

url = '%s?fields=%s&access_token=%s' % \
    (base_url, fields, ACCESS_TOKEN,)

# This API is HTTP-based and could be requested in the browser,
# with a command line utlity like curl, or using just about
# any programming language by making a request to the URL.
# Click the hyperlink that appears in your notebook output
# when you execute this code cell to see for yourself...
print(url)

# Interpret the response as JSON and convert back
# to Python data structures
content = requests.get(url).json()

# Pretty-print the JSON and display it
print(json.dumps(content, indent=1))

https://graph.facebook.com/me?fields=id,name,friends.limit(10).fields(likes.limit(10))&access_token=EAACnHoiQvFQBADx5OwQwREzHdGRD8R2OTSrFBNo6DtKv5CnvIeYDS3VIMlHHZAC9TbwtRZAxEez3iuDJ61AfN9K0ZCUNKn4tsHPrsGUWcwJ6wQwBy4IygOQcrr0b2cvOLHxBqYQcjshZCZAhFMf9qu6cSMXgaYcj9TmzYmPYWtjxZBFxrnGjp0HPxImbbesJJQ5wk4ctLVhQZDZD
{
 "id": "10157582626180199",
 "name": "Kokil Jaidka"
}


Note: If you attempt to run a query for all of your friends' likes and it appears to hang, it is probably because you have a lot of friends who have a lot of likes. If this happens, you may need to add limits and offsets to the fields in the query as described in Facebook's [field expansion](https://developers.facebook.com/docs/reference/api/field_expansion/) documentation. However, the <code>facebook</code> library that we'll use in the next example handles some of these issues, so it's recommended that you hold off and try it out first. This initial example is just to illustrate that Facebook's API is built on top of HTTP.

A couple of field limit/offset examples that illustrate the possibilities follow:

<code>
fields = 'id,name,friends.limit(10).fields(likes)'            # Get all likes for 10 friends 
fields = 'id,name,friends.offset(10).limit(10).fields(likes)' # Get all likes for 10 more friends 
fields = 'id,name,friends.fields(likes.limit(10))'            # Get 10 likes for all friends 
fields = 'id,name,friends.fields(likes.limit(10))'            # Get 10 likes for all friends
</code>

## Example 2. Querying the Graph API with Python

In [5]:
#import facebook # 
!pip install facebook-sdk
import facebook
import json

# A helper function to pretty-print Python objects as JSON

def pp(o): 
    print(json.dumps(o, indent=1))

# Create a connection to the Graph API with your access token

g = facebook.GraphAPI(ACCESS_TOKEN)

# Execute a few sample queries

print('---------------')
print('Me')
print('---------------')
pp(g.get_object('me'))
print
print('---------------')
print('My Friends')
print('---------------')
pp(g.get_connections('me', 'friends'))
print
print('---------------')
print('Social Web')
print('---------------')
pp(g.request("search", {'q' : 'social web', 'type' : 'page'}))

---------------
Me
---------------
{
 "name": "Kokil Jaidka",
 "id": "10157582626180199"
}
---------------
My Friends
---------------
{
 "data": []
}
---------------
Social Web
---------------
{
 "data": []
}


## Example 3. Results for a Graph API query for Mining the Social Web

In [70]:
# Get an instance of Mining the Social Web
# Using the page name also works if you know it.
# e.g. 'MiningTheSocialWeb' or 'CrossFit'
mtsw_id = '146803958708175'
pp(g.get_object(mtsw_id,fields='id,name,likes,engagement'))
#https://graph.facebook.com/FANPAGE_ID?access_token=ACCESS_TOKEN&fields=name,likes


{
 "name": "Mining the Social Web",
 "id": "146803958708175",
 "likes": 2568,
 "engagement": {
  "social_sentence": "2.5K people like this.",
  "count": 2568
 }
}


## Example 4. Querying the Graph API for Open Graph objects by their URLs

In [47]:
# MTSW catalog link
pp(g.get_object('http://shop.oreilly.com/product/0636920030195.do'))

# PCI catalog link
pp(g.get_object('http://shop.oreilly.com/product/9780596529321.do'))

{
 "id": "http://shop.oreilly.com/product/0636920030195.do"
}
{
 "id": "http://shop.oreilly.com/product/9780596529321.do"
}


## Example 5. Comparing likes between Coke and Pepsi fan pages

In [76]:
# Find Pepsi and Coke in search results

#pp(g.request('search', {'q' : 'pepsi', 'type' : 'page', 'limit' : 5}))
#pp(g.request('search', {'q' : 'coke', 'type' : 'page', 'limit' : 5}))

# Use the ids to query for likes

pepsi_id = '56381779049' # Could also use 'PepsiUS'
coke_id = '40796308305'  # Could also use 'CocaCola'
#print("Pepsi likes:", g.get_object(pepsi_id,fields='likes'))
# A quick way to format integers with commas every 3 digits
def int_format(n): return "{:,}".format(n)
print("Pepsi likes:", int_format(g.get_object(pepsi_id,fields='likes')['likes']))
print("Coke likes:", int_format(g.get_object(coke_id,fields='likes')['likes']))

Pepsi likes: 37,340,189
Coke likes: 106,033,240


## Example 6. Querying a page for its "feed" and "links" connections

In [16]:
pp(g.get_connections(pepsi_id, 'feed'))
pp(g.get_connections(pepsi_id, 'links'))

pp(g.get_connections(coke_id, 'feed'))
pp(g.get_connections(coke_id, 'links'))

{
 "data": [
  {
   "message": "One word: delicious.",
   "id": "56381779049_10156032126424050",
   "created_time": "2017-10-11T21:00:10+0000"
  },
  {
   "message": "Because what's more Fall than football (besides pumpkin spice)\u2014jump into this episode of The Fun Doesn't End Zone now pep.si/TFDEZ",
   "id": "56381779049_10211053156110970",
   "created_time": "2017-10-10T18:00:00+0000"
  },
  {
   "message": "When you realize we're in the best month of baseball",
   "id": "56381779049_10156026707124050",
   "created_time": "2017-10-09T22:00:28+0000"
  },
  {
   "message": "This tee is ready for all the errands (or lounging) this Saturday has in store. http://pep.si/2xYKPzF",
   "id": "56381779049_10156020164209050",
   "created_time": "2017-10-07T17:00:09+0000"
  },
  {
   "message": "Yes, we agree every good tailgate needs Pepsi. No, we did not tell them they had to say it \ud83d\ude09 pep.si/TFDEZ",
   "id": "56381779049_10210951442208186",
   "created_time": "2017-10-05T19:15:00

GraphAPIError: (#12) links field is deprecated for versions v2.4 and higher

## Example 7. Querying for all of your friends' likes

In [33]:
# First, let's query for all of the likes in your social
# network and store them in a slightly more convenient
# data structure as a dictionary keyed on each friend's
# name. We'll use a dictionary comprehension to iterate
# over the friends and build up the likes in an intuitive
# way, although the new "field expansion" feature could 
# technically do the job in one fell swoop as follows:
#
# g.get_object('me', fields='id,name,friends.fields(id,name,likes)')
#
# See Appendix C for more information on Python tips such as
# dictionary comprehensions

friends = g.get_connections("me", "friends")['data']

likes = { friend['name'] : g.get_connections(friend['id'], "likes")['data'] 
          for friend in friends }

print(likes)

{}
